In [1]:
import re
import pandas as pd
import numpy as np
from datetime import datetime
import xml.etree.ElementTree as ET

In [2]:
ns = {'xml': 'http://www.w3.org/XML/1998/namespace',
      'dflt': 'http://www.tei-c.org/ns/1.0',
      'frus':'http://history.state.gov/frus/ns/1.0',
      'xi':'http://www.w3.org/2001/XInclude'
      }

In [93]:
volume = 'frus1969-76v30.xml'
tree = ET.parse('volumes/frus1969-76v30.xml')
root = tree.getroot()

In [ ]:
# year: frus:doc-dateTime-max
# sent from: <placeName>New York</placeName>
# volume from file name
# from: <gloss type="from"> or <persName type="from">
# to: <gloss type="to"> or <persName type="to">
# source: <... type="source" ..>

# ignore all <note/>

In [ ]:
# ENTITY("PERSON")

person_df = pd.DataFrame(columns=['id','name','description'])

def extract_person(person_item):
    persName_item = person_item.find('.//dflt:persName', ns)
    person_name = persName_item.text
    person_id = persName_item.attrib['{http://www.w3.org/XML/1998/namespace}id']

    all_text = "".join(person_item.itertext())
    end_idx = all_text.find(person_name) + len(person_name+',')
    person_descp = " ".join(all_text[end_idx:].split())

    person_name = " ".join(re.sub(',',''," ".join(person_name.split(', ')[::-1])).split())

    #person_id = volume[:-4] + '_' + person_id

    global person_df
    person_df = pd.concat((person_df, pd.DataFrame({'id':[person_id],
                                                'name':[person_name],
                                                'description':[person_descp]})),ignore_index=True)
    return


persons_section = root.find("./dflt:text/dflt:front//dflt:div[@xml:id='persons']", ns)
for item in persons_section.findall('.//dflt:item', ns):
    extract_person(item)

person_df.to_csv('tables/person_single_volume.csv')

In [ ]:
# ENTITY("YEAR")

year_df = pd.DataFrame({'year':np.arange(1861,1982)})
year_df.to_csv('tables/year.csv')

In [96]:
# ENTITY("DOCUMENT")

person_df = pd.DataFrame(columns=['id','volume','subtype','date','year','title','source','person_sentby',
                                  'person_sentto','city','era','inst_sentby','inst_sentto',
                                  'person_mentioned','inst_mentioned'])

docs = root.findall('./dflt:text/dflt:body//dflt:div[@type="document"]', ns)

# extract head as well.

In [25]:
from datetime import datetime

# id
id = volume[:-4] + '_' + docs[0].attrib['{http://www.w3.org/XML/1998/namespace}id']

# subtype
subtype = docs[0].attrib['subtype']

# date and year
if subtype!='editorial-note':
    fmt = docs[0].attrib['{http://history.state.gov/frus/ns/1.0}doc-dateTime-max']
    date = datetime.strptime(fmt.split('T')[0], '%Y-%m-%d')
    year = datetime.strptime(fmt.split('T')[0], '%Y-%m-%d').year

# source
source_tag = docs[0].find('.//dflt:note[@type="source"]',ns)
source = " ".join(ET.tostring(source_tag, encoding='unicode', method='text').split())

# title
head_tag = docs[0].find('./dflt:head', ns)
child_note_tags = head_tag.findall('./dflt:note', ns)

for note_tag in child_note_tags:
    head_tag.remove(note_tag)

title = " ".join(ET.tostring(head_tag, encoding='unicode', method='text').split())

# city
city = docs[0].find('.//dflt:placeName',ns).text

# person_sentby
person_sentby = []

# persName
for pers_tag in docs[0].findall('.//dflt:persName[@type="from"]',ns):
    if pers_tag is not None:
        if 'corresp' in pers_tag.attrib:
            person_sentby.append(pers_tag.attrib['corresp'])
        else:
            person_sentby.append(pers_tag.text)

# list -not included yet-
#docs[0].findall('.//dflt:list',ns)[0].attrib


# signed
signed_person_tag = docs[0].find('.//dflt:signed//dflt:persName',ns)
if signed_person_tag is not None:
    if 'corresp' in signed_person_tag.attrib:
        person_sentby.append(signed_person_tag.attrib['corresp'])
    else:
        person_sentby.append(signed_person_tag.text)

# person_sentto
person_sentto = []

# persName
for pers_tag in docs[0].findall('.//dflt:persName[@type="to"]',ns):
    if pers_tag is not None:
        if 'corresp' in pers_tag.attrib:
            person_sentto.append(pers_tag.attrib['corresp'])
        else:
            person_sentto.append(pers_tag.text)

# list -not included yet-
#docs[0].findall('.//dflt:list[@type="to"]',ns)[0].attrib

1973-03-30 00:00:00
1973


In [ ]:
# ENTITY("ERA")

## done above ^
## construction below 

In [ ]:
# this function handles file's sender and receiver
def process_from_to_types(from_list):

    processed_list = []

    for item in from_list:
        
        if item.tag == 'persName' and 'corresp' in item.attrib.keys():
            person_id = item.attrib['corresp']
            processed_list.append(person_id)
        elif item.tag == 'persName':
            processed_list.append(item.text)
        elif item.tag == 'gloss':
            processed_list.append(item.text)
        else:
            raise NotImplementedError('from type tag unidentified')
    
    return processed_list

In [ ]:
subseries_id = root.find('teiHeader/fileDesc/titleStmt/title[@type="sub-series"]').text
volume_id = root.find('teiHeader/fileDesc/titleStmt/title[@type="volume-number"]').text

doc_dict = {}

def process_document(doc_elem):

    doc_id = doc_elem.attrib['{http://www.w3.org/XML/1998/namespace}id']
    doc_date = doc_elem.attrib["{http://history.state.gov/frus/ns/1.0}doc-dateTime-max"].split('T')[0]
    doc_date = datetime.strptime(doc_date, '%Y-%m-%d')

    doc_sent_place = doc_elem.find('.//placeName')
    if doc_sent_place is not None:
        doc_sent_place = doc_sent_place.text

    from_list = doc_elem.findall('head//*[@type="from"]')
    to_list = doc_elem.findall('head//*[@type="to"]')

    doc_sent_from = process_from_to_types(from_list)
    doc_sent_to = process_from_to_types(to_list)

    doc_source = doc_elem.find('.//*[@type="source"]')
    if doc_source is not None:
        doc_source = " ".join("".join(doc_source.itertext()).split())

    doc_dict["-".join([subseries_id,volume_id,doc_id])] = {'subseries':subseries_id, 'volume':volume_id, 'doc_id':doc_id,'date':doc_date, 'place':doc_sent_place, 'from':doc_sent_from, 'to':doc_sent_to, 'source':doc_source}

    return

for doc_elem in root.findall('text/body/div//div[@type="document"]'):
    process_document(doc_elem)